In [1]:
import pandas as pd
import numpy as np
import pystan

In [2]:
firewood_code = """

data {
    // samples per row by species
    int<lower=0> totals1[3];
    int<lower=0> totals2[3];
    int<lower=0> totals3[3];
    int<lower=0> totals4[3];
    
    // ready to burn samples per row by species
    int<lower=0> burnables1[3];
    int<lower=0> burnables2[3];
    int<lower=0> burnables3[3];
    int<lower=0> burnables4[3];
}

parameters {
    // probabilities per row of each species
    vector<lower=0.0, upper=1.0>[3] p_species1;
    vector<lower=0.0, upper=1.0>[3] p_species2;
    vector<lower=0.0, upper=1.0>[3] p_species3;
    vector<lower=0.0, upper=1.0>[3] p_species4;
    
    // probabilities per row per species of burnable
    vector<lower=0.0, upper=1.0>[3] p_burnable1;
    vector<lower=0.0, upper=1.0>[3] p_burnable2;
    vector<lower=0.0, upper=1.0>[3] p_burnable3;
    vector<lower=0.0, upper=1.0>[3] p_burnable4;
}

model {
    // row 1 samples by species
    p_species1[1] ~ beta(5, 5); // row 1 alder prior
    p_species1[2] ~ beta(2, 8); // row 1 birch prior
    p_species1[3] ~ beta(3, 7); // row 1 pine prior
    totals1 ~ multinomial(softmax(p_species1));
    
    // row 2 samples by species
    p_species2[1] ~ beta(7, 3); // row 2 alder prior
    p_species2[2] ~ beta(2, 8); // row 2 birch prior
    p_species2[3] ~ beta(1, 9); // row 2 pine prior
    totals2 ~ multinomial(softmax(p_species2));
    
    // row 3 samples by species
    p_species3[1] ~ beta(15, 5); // row 3 alder prior
    p_species3[2] ~ beta(4, 16); // row 3 birch prior
    p_species3[3] ~ beta(1, 19); // row 3 pine prior
    totals3 ~ multinomial(softmax(p_species3));
    
    // row 4 samples by species
    p_species4[1] ~ beta(15, 5); // row 4 alder prior
    p_species4[2] ~ beta(4, 16); // row 4 birch prior
    p_species4[3] ~ beta(1, 19); // row 4 pine prior
    totals4 ~ multinomial(softmax(p_species4));
    
    // separate binomial distributions per row per species for number of burnable logs
    for (i in 1:3) {
        burnables1[i] ~ binomial(totals1[i], p_burnable1[i]);
        burnables2[i] ~ binomial(totals2[i], p_burnable2[i]);
        burnables3[i] ~ binomial(totals3[i], p_burnable3[i]);
        burnables4[i] ~ binomial(totals4[i], p_burnable4[i]);
    }
}

generated quantities {
    vector[3] percent_species;
    vector[3] percent_burnable;
    real total_percent_burnable;
    real estimated_BTUs;
    real days_of_fuel;
    
    percent_species = (softmax(p_species1) + softmax(p_species2) + softmax(p_species3) + softmax(p_species4))/4;\
    
    for (i in 1:3) {
        percent_burnable[i] = (softmax(p_species1)[i]*p_burnable1[i] + softmax(p_species2)[i]*p_burnable2[i] + softmax(p_species3)[i]*p_burnable3[i] + softmax(p_species4)[i]*p_burnable4[i])/4;
    }
    total_percent_burnable = sum(percent_burnable);
    
    estimated_BTUs = (percent_burnable[1]*19000000 + percent_burnable[2]*23600000 + percent_burnable[3]*20500000)*1.124;
    days_of_fuel = estimated_BTUs/250000*0.77;
    
}
"""

firewood_model = pystan.StanModel(model_code=firewood_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ea83cc44d3204fc9828c6df446b59bb0 NOW.


In [3]:
# total count per each species
df = pd.read_csv('firewood.csv')
df['row'] = 'totals' + df['row'].astype(str)
firewood_data = pd.pivot_table(df, index='species', columns='row', aggfunc='count').fillna(0).astype(int).droplevel(0, axis=1).to_dict(orient='list')

# burnable counts per each species
df = pd.read_csv('firewood.csv')
df['row'] = 'burnables' + df['row'].astype(str)
df.loc[(df['moisture'] <= 0.2) | ((df['moisture'] <= 0.23) & (df['species'] == 'alder')), 'n'] = 1
firewood_data.update(pd.pivot_table(df, index='species', columns='row', values='n', aggfunc=np.sum).fillna(0).astype(int).to_dict(orient='list'))

# run model
firewood_model.sampling(data=firewood_data, pars=['percent_species', 'total_percent_burnable', 'days_of_fuel'], chains=4, iter=100000, warmup=5000)

Inference for Stan model: anon_model_ea83cc44d3204fc9828c6df446b59bb0.
4 chains, each with iter=100000; warmup=5000; thin=1; 
post-warmup draws per chain=95000, total post-warmup draws=380000.

                         mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
percent_species[1]       0.47  2.2e-5   0.02   0.43   0.46   0.47   0.48    0.5 503787    1.0
percent_species[2]       0.28  1.8e-5   0.01   0.26   0.27   0.28   0.29   0.31 507542    1.0
percent_species[3]       0.25  1.5e-5   0.01   0.23   0.25   0.25   0.26   0.28 502469    1.0
total_percent_burnable   0.44  7.2e-5   0.05   0.34    0.4   0.44   0.47   0.54 488654    1.0
days_of_fuel            30.47  5.1e-3   3.57  23.61  28.01  30.43  32.87  37.56 491110    1.0
lp__                   -207.0  9.8e-3    3.8 -215.4 -209.4 -206.7 -204.3 -200.7 151293    1.0

Samples were drawn using NUTS at Sun Nov 29 12:45:20 2020.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the p